In [1]:
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [2]:
import zarr
from candidate_graph import create_candidate_graph
import daisy
import linajea

In [3]:
candidate_graph = create_candidate_graph('anno_alice_T2030_tiny.zarr')

/
 ├── Fragment_stats
 │   ├── Position
 │   ├── Volume
 │   └── id
 ├── Fragments (3, 15, 30, 30) uint64
 ├── Merge_tree
 │   ├── Merge
 │   └── Scoring
 ├── Raw (4, 3, 15, 30, 30) float64
 └── gt_trackimage (3, 15, 30, 30) uint64
add 35321834118144 985274108615169 count 0.03571428571428571
 iterating time:  0.0315396785736084
add 35321834118144 1196380341148161 count 0.029411764705882353
 iterating time:  0.034295082092285156
add 35321834118144 27725491381213185 count 0.0012690355329949238
 iterating time:  0.06362557411193848
add 35321834118144 462041294558989825 count 7.614986293024673e-05
 iterating time:  0.7636725902557373
add 35321834118144 462920903861210625 count 7.60051683514479e-05
 iterating time:  0.8359782695770264
add 35321834118144 470696650092841985 count 7.474958887726118e-05
 iterating time:  0.9082634449005127
add 35321834118144 472702159301905921 count 7.44324525493115e-05
 iterating time:  0.9828569889068604
add 35321834118144 473229924883238401 count 7.434944237

In [4]:
G = candidate_graph

In [8]:
from funlib.math import encode64
encode64((1,7,14,14,40500),bits=[9,12,12,12,19])

1424967189886144001

In [14]:
import networkx as nx
g = nx.read_gexf('text.gexf')

for node in g.nodes:
    if  g.nodes[node]['parent'] == node:
        print(g.nodes[node])

In [50]:
from linajea_cost_test import get_merge_graph ,get_merge_graph_from_array
t = 0
z = zarr.open('anno_alice_T2030_tiny.zarr','r') 
merge_tree = z['Merge_tree/Merge/'+str(t)]
scores = z['Merge_tree/Scoring/'+str(t)]
merge_tree = get_merge_graph_from_array(merge_tree,scores)

g = get_merge_graph(G,0)

In [61]:
g = get_merge_graph(G,0)


In [60]:


def get_merge_graph(candidate_graph, t):
    """Get merge graph from Linajea CandidateGraph
    The nodes can be chosen for that time point.
    The edges can be obtained from the "parent" attribute of the node.
    Parameters
    ----------
    candidate_graph: linajea.CandidateGraph
        The Candidate graph containing all the nodes of interest
    t: int
        The time to consider
    """
    g = nx.DiGraph()
    nodes = [nid for nid in candidate_graph.nodes()
                if candidate_graph.nodes[nid]['t'] == t]
    # Iteratively add nodes
    for nid in nodes:
        node = candidate_graph.nodes[nid]
        if nid not in g:
            g.add_node(nid,**node)
        if node['parent'] != nid and node['id']:   
        # determine a root as node['parent'] = nid in candidate_graph func add_nodes_from_merge_tree()
            g.add_edge(nid, node['parent'])
    return g

In [47]:
nodes = [nid for nid in candidate_graph.nodes()
                if candidate_graph.nodes[nid]['t'] == 1]

In [43]:
G.nodes(data=True)

NodeDataView({19140315621363712: {'t': 0, 'z': 8, 'y': 12, 'x': 2, 'score': 0.04924517869949341, 'parent': 53339542484161024, 'id': 19140315621363712}, 8584995398486528: {'t': 0, 'z': 5, 'y': 9, 'x': 1, 'score': 0.0001, 'parent': 19140315621363712, 'id': 8584995398486528}, 10555337427916288: {'t': 0, 'z': 11, 'y': 15, 'x': 3, 'score': 0.0001, 'parent': 19140315621363712, 'id': 10555337427916288}, 53339542484161024: {'t': 0, 'z': 9, 'y': 18, 'x': 4, 'score': 0.08333110809326172, 'parent': 54465442391003648, 'id': 53339542484161024}, 34199252666160128: {'t': 0, 'z': 10, 'y': 22, 'x': 5, 'score': 0.0001, 'parent': 53339542484161024, 'id': 34199252666160128}, 15129503355377664: {'t': 0, 'z': 12, 'y': 9, 'x': 26, 'score': 0.11383092403411865, 'parent': 17944253120387072, 'id': 15129503355377664}, 12420306700342784: {'t': 0, 'z': 11, 'y': 7, 'x': 26, 'score': 0.0001, 'parent': 15129503355377664, 'id': 12420306700342784}, 2709428612635136: {'t': 0, 'z': 13, 'y': 16, 'x': 27, 'score': 0.0001, 

In [30]:
roots = [x for x in g.nodes if g.in_degree(x) == 0]

In [18]:
# check the root nodes
import networkx as nx
for node in G.nodes:
    if  G.nodes[node]['parent'] is node:
        print(G.nodes[node])
# nodes with no parent is the root of merge_tree

{'t': 0, 'z': 7, 'y': 14, 'x': 14, 'score': 0.9081573486328125, 'parent': 474989143487680000, 'id': 474989143487680000}
{'t': 2, 'z': 7, 'y': 14, 'x': 14, 'score': 0.9649198055267334, 'parent': 474989143487680002, 'id': 474989143487680002}


In [96]:
roi = daisy.Roi((0, 0, 0, 0), (3, 15, 25, 25))

    # input graph
graph = linajea.tracking.TrackGraph(G, frame_key='t', roi=roi)

In [97]:
from linajea_cost_test import get_edge_indicator_costs, get_node_indicator_costs, get_constraints


import time
import logging

logger = logging.getLogger(__name__)


class TrackingConfig():
    def __init__(self, solve_config):
        self.solve = solve_config

In [98]:
ps = {
            "track_cost": 4.0,
            "weight_edge_score": -0.1,
            "weight_node_score": -0.1,
            "selection_constant": -1.0,
            "max_cell_move": 0.0,
            "block_size": [5, 100, 100, 100],
            }
job = {"num_workers": 5, "queue": "normal"}

solve_config = linajea.config.SolveConfig(parameters=ps,
                                            job=job,
                                            context=[2, 100, 100, 100])

solve_config.solver_type = None
solve_config.timeout = 100000

In [ ]:
config = TrackingConfig(solve_config)
constrs = get_constraints()

linajea.tracking.track(
                graph,
                config,
                frame_key='t',
                selected_key='selected',
                edge_indicator_costs=get_edge_indicator_costs,
                node_indicator_costs=get_node_indicator_costs,
                constraints_fns=constrs[0],
                pin_constraints_fns=constrs[1]
                )
